In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pytorch_lightning as pl

import random as rand
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import defaultdict

In [6]:
torch.manual_seed(28)
pl.trainer.seed_everything(seed=5)
trainset = dsets.CIFAR10('./data',train= True, download= True, transform= transforms.ToTensor())
testset = dsets.CIFAR10('./data', train = False, download= True, transform= transforms.ToTensor())
batch_size = 100
train_data = torch.utils.data.DataLoader(dataset=trainset,
                                           batch_size= batch_size, shuffle=True)
test_data = torch.utils.data.DataLoader(dataset=testset,
                                           batch_size= batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
'''
For Multiple optimizers
'''

class LightningLogisticClassifier(pl.LightningModule):
    def __init__(self,optim_name,act_func):
        super(LightningLogisticClassifier, self).__init__()
        # CIFAR-10 images are (3, 32, 32) (channels, width, height) 
        self.linear1 = nn.Linear(32*32*3, 2048)
        self.linear2 = nn.Linear(2048, 10)
        self.drop1 = nn.Dropout(0.2)
        self.loss_function = nn.CrossEntropyLoss()
        self.total_loss = []
        
        if(act_func=='relu'):
            self.activation = nn.ReLU()
        elif(act_func=='tanh'):
            self.activation = nn.Tanh() 
        elif(act_func=='sigmoid'):
            self.activation = nn.Sigmoid()
        
        
        self.seq_model = nn.Sequential(self.linear1,self.drop1,self.activation,self.linear2)
        
        if (optim_name=='sgd'):
            self.optimizer = torch.optim.SGD(self.parameters(), lr=1e-3,momentum=0.9)
        elif (optim_name=='adam'):
            self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        elif (optim_name == 'adadelta'):
            self.optimizer = torch.optim.Adadelta(self.parameters(), lr=1e-3)#,weight_decay=0)
        elif(optim_name=='rmsprop'):
            self.optimizer = torch.optim.RMSprop(self.parameters(), lr=0.01) #, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0)
        elif(optim_name =='adagrad'):
            self.optimizer =torch.optim.Adagrad(self.parameters(), lr=0.01)#,weight_decay=0)    
        
        
    def forward(self, x):
        batch_size, channels, width, height = x.size()
        x = x.view(batch_size, -1)
        
        output = self.seq_model(x)
        return output      
        

    def cross_entropy_loss(self, outputs, labels):
        return self.loss_function(outputs, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)

        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.cross_entropy_loss(y_hat, y)
        
        return {'test_loss': loss}
    
    def test_epoch_end(self,outputs):
        test_loss=torch.stack([x['test_loss'] for x in outputs]).mean()
        log= {'avg_test_loss':test_loss}
        
        return {'log':log}
    
    
    def configure_optimizers(self):
        return self.optimizer
        
   
    def train_dataloader(self):
        return train_data
    
    def test_dataloader(self):
        return test_data
    

In [20]:
if __name__=='__main__':
    train_data = torch.utils.data.DataLoader(dataset=trainset,
                                           batch_size= batch_size,  num_workers=4,shuffle=True)
    test_data = torch.utils.data.DataLoader(dataset=testset,
                                           batch_size= batch_size, num_workers=4)
    
    optimizers=['sgd','adam','adadelta','rmsprop','adagrad']
    acti_functions=['relu','tanh','sigmoid']

    for opt in optimizers:
        for activ in acti_functions:
            print("Activation Function: {} with Optimizer:{}".format(str(activ),str(opt)))
            # train
            model = LightningLogisticClassifier(optim_name=opt,act_func= activ)
            #trainer =pl.Trainer(fast_dev_run=True) 
            trainer =pl.Trainer(max_nb_epochs=2) 
            trainer.fit(model,train_dataloader=train_data)
            trainer.test(test_dataloaders=test_data)
      

GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name          | Type             | Params
-----------------------------------------------
0 | linear1       | Linear           | 6 M   
1 | linear2       | Linear           | 20 K  
2 | relu          | ReLU             | 0     
3 | drop1         | Dropout          | 0     
4 | loss_function | CrossEntropyLoss | 0     
5 | seq_model     | Sequential       | 6 M   


c:\users\shunn0\appdata\local\programs\python\python37\lib\site-packages\pytorch_lightning\utilities\distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)


GPU available: False, used: False


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(1.8229)}
--------------------------------------------------------------------------------



No environment variable for node rank defined. Set as 0.

  | Name          | Type             | Params
-----------------------------------------------
0 | linear1       | Linear           | 6 M   
1 | linear2       | Linear           | 20 K  
2 | relu          | ReLU             | 0     
3 | drop1         | Dropout          | 0     
4 | loss_function | CrossEntropyLoss | 0     
5 | seq_model     | Sequential       | 6 M   


...

--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(2.1612)}
--------------------------------------------------------------------------------




  | Name          | Type             | Params
-----------------------------------------------
0 | linear1       | Linear           | 6 M   
1 | linear2       | Linear           | 20 K  
2 | relu          | ReLU             | 0     
3 | drop1         | Dropout          | 0     
4 | loss_function | CrossEntropyLoss | 0     
5 | seq_model     | Sequential       | 6 M   


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(2.3028)}
--------------------------------------------------------------------------------



GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name          | Type             | Params
-----------------------------------------------
0 | linear1       | Linear           | 6 M   
1 | linear2       | Linear           | 20 K  
2 | relu          | ReLU             | 0     
3 | drop1         | Dropout          | 0     
4 | loss_function | CrossEntropyLoss | 0     
5 | seq_model     | Sequential       | 6 M   


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_loss': tensor(1.6786)}
--------------------------------------------------------------------------------



In [2]:
# Start tensorboard.
%load_ext tensorboard

%tensorboard --logdir lightning_logs_06_20/ --host localhost


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 13032), started 0:13:48 ago. (Use '!kill 13032' to kill it.)